In [4]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from utils.train_immagini import train_model
from utils.dataset_immagini import ImmaginiDataset
from models.model_immagini import CNNModel

if __name__ == "__main__":
    data_dir = './data'
    batch_size = 32
    img_size = 224
    num_epochs = 10
    learning_rate = 0.001
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    train_dataset = ImmaginiDataset(data_dir, split="train", img_size=img_size)
    val_dataset = ImmaginiDataset(data_dir, split="val", img_size=img_size)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model = CNNModel(num_classes=len(train_dataset.classes)).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    train_model(train_loader, val_loader, model, criterion, optimizer, num_epochs=num_epochs, device=device)

Epoch [1/10], Train Loss: 1.5062, Train Acc: 53.15%, Val Loss: 0.4350, Val Acc: 85.11%
Epoch [2/10], Train Loss: 0.1793, Train Acc: 93.24%, Val Loss: 0.1628, Val Acc: 92.55%
Epoch [3/10], Train Loss: 0.2458, Train Acc: 92.54%, Val Loss: 0.5563, Val Acc: 82.98%
Epoch [4/10], Train Loss: 0.1750, Train Acc: 92.31%, Val Loss: 0.1237, Val Acc: 95.74%
Epoch [5/10], Train Loss: 0.0315, Train Acc: 98.83%, Val Loss: 0.0358, Val Acc: 98.94%
Epoch [6/10], Train Loss: 0.0157, Train Acc: 99.77%, Val Loss: 0.0249, Val Acc: 98.94%
Epoch [7/10], Train Loss: 0.0062, Train Acc: 99.77%, Val Loss: 0.0040, Val Acc: 100.00%
Epoch [8/10], Train Loss: 0.0024, Train Acc: 100.00%, Val Loss: 0.0108, Val Acc: 100.00%
Epoch [9/10], Train Loss: 0.0017, Train Acc: 100.00%, Val Loss: 0.0176, Val Acc: 98.94%
Epoch [10/10], Train Loss: 0.0006, Train Acc: 100.00%, Val Loss: 0.0033, Val Acc: 100.00%
Best Validation Accuracy: 100.00%
